In [1]:
import sys
import numpy as np
import cv2
import os
from PIL import Image

sys.path.append("/app")

In [2]:
# Lista de classes fornecidas pelo usuário
classes = [
    "Skin", "Out of focus area", "Oral mucosa", "Enamel", "Tongue", "Lip", "Hard palate",
    "Specular reflection", "Attached gingiva", "Soft palate", "Hair", "Marginal gingiva",
    "Prosthetics", "Shadow/Noise", "Plastic", "Metal", "Gingivitis", "Attrition/Erosion",
    "Inflammation", "Pigmentation", "Calculus", "Initial caries", "Stain", "Fluorosis",
    "Microfracture", "Root", "Plaque", "Dentine caries", "Ulcer", "Leukoplakia",
    "Blood vessel", "Mole", "Malignant lesion", "Fibroma", "Makeup"
]

# Criar um dicionário mapeando o nome da classe para um valor numérico (int)
class_map = {cls: idx + 1 for idx, cls in enumerate(classes)}  # Começa em 1 (0 = fundo)

In [3]:
def parse_annotation_mask_id(filepath, width, height, class_map):
    """Gera uma máscara com IDs inteiros para treinamento"""
    mask = np.zeros((height, width), dtype=np.uint8)

    with open(filepath, "r", encoding="utf-8") as f:
        for line in f:
            parts = line.strip().split(";")
            if len(parts) < 3:
                continue

            class_name = parts[0].strip()
            class_id = class_map.get(class_name)
            if class_id is None:
                continue

            points = []
            for coord in parts[2:]:
                try:
                    x, y = map(int, coord.strip().split(","))
                    if 0 <= x < width and 0 <= y < height:
                        points.append([x, y])
                except:
                    continue

            if points:
                contour = np.array(points, dtype=np.int32).reshape((-1, 1, 2))
                cv2.fillPoly(mask, [contour], color=class_id)

    return mask


In [4]:
def parse_annotation_mask_rgb(filepath, width, height):
    """Gera uma máscara colorida RGB para visualização"""
    mask = np.zeros((height, width, 3), dtype=np.uint8)

    with open(filepath, "r", encoding="utf-8") as f:
        for line in f:
            parts = line.strip().split(";")
            if len(parts) < 3:
                continue

            try:
                r, g, b = map(int, parts[1].strip().split(","))
                color = (r, g, b)
            except:
                continue

            points = []
            for coord in parts[2:]:
                try:
                    x, y = map(int, coord.strip().split(","))
                    if 0 <= x < width and 0 <= y < height:
                        points.append([x, y])
                except:
                    continue

            if points:
                contour = np.array(points, dtype=np.int32).reshape((-1, 1, 2))
                cv2.fillPoly(mask, [contour], color=color)

    return mask


In [5]:


filepath = "/app/dataset/Specim_IQ/front/0aa155fa_front_2_annot.csv"
# Para treinamento
mask_id = parse_annotation_mask_id(filepath, 513, 513, class_map)
cv2.imwrite("mask_id.png", mask_id)  # ou use Image.fromarray(mask_id).save(...)

# Para visualização
mask_rgb = parse_annotation_mask_rgb(filepath, 513, 513)
cv2.imwrite("mask_rgb.png", mask_rgb)


True

In [6]:
# Caminho base
base_dir = "/app/dataset"

# Definição das resoluções por câmera
camera_resolutions = {
    "Specim_IQ": (512, 512),
    "Nuance_EX": (1392, 1040)
}

# Log de processamento
log = []

# Percorrer diretórios
for root, _, files in os.walk(base_dir):
    for file in files:
        if file.endswith("_annot.csv"):
            annot_path = os.path.join(root, file)
            base_name = file.replace("_annot.csv", "")
            image_name = base_name + ".tif"
            image_path = os.path.join(root, image_name)

            # Detecta qual câmera está no caminho
            resolution = None
            for cam_name, (w, h) in camera_resolutions.items():
                if cam_name in root:
                    resolution = (w, h)
                    break

            if resolution is None:
                log.append(("UNKNOWN_CAMERA", file))
                continue

            width, height = resolution
            try:
                mask = parse_annotation_mask_id(annot_path, width, height, class_map)
                out_path = os.path.join(root, base_name + "_masks.png")
                Image.fromarray(mask).save(out_path)
                log.append(("OK", file))
            except Exception as e:
                log.append(("ERROR", file, str(e)))


# import pandas as pd
# log_df = pd.DataFrame(log, columns=["Status", "Arquivo", "Mensagem"][:len(log[0])])
# import ace_tools as tools; tools.display_dataframe_to_user(name="Log de Geração de Máscaras", dataframe=log_df)
